# Hochschule Bonn-Rhein-Sieg

# Neural Networks, WS17/18

# Assignment 09 (04-December-2017)

##  Debaraj Barua, Naresh Kumar Gurulingan

In [1]:
import numpy as np

## Question 1:

Mindmap of chapter 9:

---
## Question 2:

2) Show that in the SOM algorithm the winner neuron for an input x is that neuron k whose weight vector w_k maximizes the inner product < w_k,x > of x and wk, take x and wk as normalized.

---
## Question 3:

3) Consider the one dimensional input space S={0.1, 0.2, 0.4, 0.5}. Cluster S using a one dimensional SOM network with:

– 2 nodes.

– learning rate equal to 0.1.

– Neighborhood function which is equal to 1 for the winner neuron and is 0 otherwise.

– two Weight initializations:

-- w1=0.15, w2=0.45

-- w1=0.3, w2=0.9

– Stopping criterion: (see PDF)

Comment on the two clusterings you obtained using the two different weight initializations.

---
## Question 4:

4) Implement a SOM to solve the traveling salesman problem (TSP)